In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Some constants 
INPUT_FOLDER = '../../../../kaggle-dsb-data/sample_images/'
patients = os.listdir(INPUT_FOLDER) # patients is a list of all 20 patient ids in sample_images
patients.sort()
print(patients)

['00cba091fa4ad62cc3200a657aeb957e', '0a099f2549429d29b32f349e95fb2244', '0a0c32c9e08cc2ea76a71649de56be6d', '0a38e7597ca26f9374f8ea2770ba870d', '0acbebb8d463b4b9ca88cf38431aac69', '0b20184e0cd497028bdd155d9fb42dc9', '0bd0e3056cbf23a1cb7f0f0b18446068', '0c0de3749d4fe175b7a5098b060982a1', '0c37613214faddf8701ca41e6d43f56e', '0c59313f52304e25d5a7dcf9877633b1', '0c60f4b87afcb3e2dfa65abbbf3ef2f9', '0c98fcb55e3f36d0c2b6507f62f4c5f1', '0c9d8314f9c69840e25febabb1229fa4', '0ca943d821204ceb089510f836a367fd', '0d06d764d3c07572074d468b4cff954f', '0d19f1c627df49eb223771c28548350e', '0d2fcf787026fece4e57be167d079383', '0d941a3ad6c889ac451caf89c46cb92a', '0ddeb08e9c97227853422bd71a2a695e', '0de72529c30fe642bc60dcb75c87f6bd']


In [2]:
# Load the scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)] # slices becomes a list of all dicom images for each patient
    slices.sort(key = lambda x: int(x.InstanceNumber)) # sort all of the slices by their InstanceNumber (which determines their order)
    try: # try to find z location of the slice from ImagePositionPatient attribute
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2]) 
    except: # otherwise, subtract one slice location from the other to get the thickness
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices: # for slice, make new attribute that is the slice's thickness 
        s.SliceThickness = slice_thickness
        
    return slices

In [3]:
patient_scans = load_scan(INPUT_FOLDER + patients[0])
print(patient_scans)

[(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.840.113654.2.55.294523283581565931298765832966512471268
(0008, 0060) Modality                            CS: 'CT'
(0008, 103e) Series Description                  LO: 'Axial'
(0010, 0010) Patient's Name                      PN: '00cba091fa4ad62cc3200a657aeb957e'
(0010, 0020) Patient ID                          LO: '00cba091fa4ad62cc3200a657aeb957e'
(0010, 0030) Patient's Birth Date                DA: '19000101'
(0018, 0050) Slice Thickness                     DS: '2.5'
(0018, 0060) KVP                                 DS: ''
(0020, 000d) Study Instance UID                  UI: 2.25.86208730140539712382771890501772734277950692397709007305473
(0020, 000e) Series Instance UID                 UI: 2.25.11575877329635228925808596800269974740893519451784626046614
(0020, 0011) Series Number              

In [ ]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans]) # image = 3D numpy array where each layer is a scan
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

## Meaning of .RescaleIntercept and .RescaleSlope
Every CT image will have this attribute, and you can use them to scale the stored pixel values to hounsfield units (which will help determine the type of material for each pixel in the CT scan). Find explanations [here](https://blog.kitware.com/dicom-rescale-intercept-rescale-slope-and-itk/) and [here](http://stackoverflow.com/questions/10193971/rescale-slope-and-rescale-intercept).

In [ ]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness] + scan[0].PixelSpacing)) # normally the PixelSpacing attribute will only contain spacing for the [x, y], that is why you need to add z (or the slice thickness)
    spacing = np.array(list(spacing)) # this gives pixel spacing in terms of [z, x, y] where x=y

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape) # the issue is that if it rounds well for this case, does it round well for all the cases? if it doesnt how will you figure out the best way for it to round to the same resolution for all images if they round differently
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

In [ ]:
for patient in patients:
    patient_scan = load_scan(INPUT_FOLDER + patient) # load all the dicom scans for the patient, including the slice thickness
    patient_pixels = get_pixels_hu(patient_scan) # take patients scans, return single 3D numpy array that contains hounsfield unit pixels for each scan
    resampled_pixels, real_spacing = resample(patient_pixels, patient_scan, new_spacing=[1, 1, 1])